# LangChain + OpenRouter + Salesforce

This notebook:
- Loads secrets from `agents/python/.env.local`.
- Sets up an OpenRouter-backed LangChain `ChatOpenAI` client.
- Uses JWT to read Accounts from the RenoCrypt scratch org via REST/SOQL.
- Asks the LLM for an overview of the customer base from that snapshot.


In [1]:
from __future__ import annotations

import os
from pathlib import Path

from dotenv import load_dotenv

def find_python_root(max_up: int = 10) -> Path:
    """Locate the `agents/python` directory starting from the current working dir."""
    p = Path.cwd().resolve()
    for _ in range(max_up):
        if p.name == "python" and p.parent.name == "agents":
            return p
        candidate = p / "agents" / "python"
        if candidate.exists():
            return candidate
        p = p.parent
    raise RuntimeError("Could not locate agents/python directory from current path")

PYTHON_ROOT = find_python_root()
PROJECT_ROOT = PYTHON_ROOT.parent.parent

# Load .env then .env.local (local overrides)
env_main = PYTHON_ROOT / ".env"
env_local = PYTHON_ROOT / ".env.local"

if env_main.exists():
    load_dotenv(env_main, override=False)
if env_local.exists():
    load_dotenv(env_local, override=True)

# Support either OPENROUTER_API_KEY or OPEN_ROUTER_API
raw_key = os.getenv("OPENROUTER_API_KEY") or os.getenv("OPEN_ROUTER_API")
if not raw_key:
    raise RuntimeError("Missing OPENROUTER_API_KEY or OPEN_ROUTER_API in agents/python/.env.local")

os.environ["OPENROUTER_API_KEY"] = raw_key

print("✓ Loaded OpenRouter API key (not shown)")
print(f"Python root: {PYTHON_ROOT}")
print(f"Project root: {PROJECT_ROOT}")


✓ Loaded OpenRouter API key (not shown)
Python root: /Users/ac/dev/salesforce/ACDevHub/agents/python
Project root: /Users/ac/dev/salesforce/ACDevHub


In [2]:
from langchain_openai import ChatOpenAI

# OpenRouter-backed LLM (update model as desired)
llm = ChatOpenAI(
    model="tngtech/deepseek-r1t2-chimera:free",
    api_key=os.environ["OPENROUTER_API_KEY"],
    base_url="https://openrouter.ai/api/v1",
)

llm
# Optional sanity check (commented to avoid automatic token use):
# llm.invoke("Say a short hello from OpenRouter.")


ChatOpenAI(profile={}, client=<openai.resources.chat.completions.completions.Completions object at 0x112c74490>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x113667510>, root_client=<openai.OpenAI object at 0x11276fb90>, root_async_client=<openai.AsyncOpenAI object at 0x113667210>, model_name='tngtech/deepseek-r1t2-chimera:free', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://openrouter.ai/api/v1')

## Salesforce JWT auth + REST helpers

Reuse the same JWT flow as the other notebooks to read from the scratch org.

In [3]:
import time
import json
import urllib.parse

import httpx
import jwt

required_sf_keys = [
    "SF_CLIENT_ID",
    "SF_USERNAME",
    "SF_LOGIN_URL",
    "SF_AUDIENCE",
    "SF_JWT_KEY_PATH",
]
missing_sf = [k for k in required_sf_keys if not os.environ.get(k)]
if missing_sf:
    raise RuntimeError(f"Missing required SF env vars: {', '.join(missing_sf)}")

SF_CLIENT_ID = os.environ["SF_CLIENT_ID"]
SF_USERNAME = os.environ["SF_USERNAME"]
SF_LOGIN_URL = os.environ["SF_LOGIN_URL"].rstrip("/")
SF_AUDIENCE = os.environ["SF_AUDIENCE"]
SF_JWT_KEY_PATH = os.environ["SF_JWT_KEY_PATH"]
SF_API_VERSION = os.environ.get("SF_API_VERSION", "65.0")


def build_sf_jwt_assertion() -> str:
    key_bytes = Path(SF_JWT_KEY_PATH).read_bytes()
    now = int(time.time())
    payload = {
        "iss": SF_CLIENT_ID,
        "sub": SF_USERNAME,
        "aud": SF_AUDIENCE,
        "exp": now + 5 * 60,
    }
    token = jwt.encode(payload, key_bytes, algorithm="RS256")
    if isinstance(token, bytes):
        token = token.decode("utf-8")
    return token


def request_sf_access_token() -> tuple[str, str]:
    assertion = build_sf_jwt_assertion()
    url = f"{SF_LOGIN_URL}/services/oauth2/token"
    data = {
        "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
        "assertion": assertion,
    }
    with httpx.Client(timeout=20) as client:
        resp = client.post(url, data=data)
        resp.raise_for_status()
    body = resp.json()
    token = body["access_token"]
    instance_url = body["instance_url"]
    os.environ["SF_ACCESS_TOKEN"] = token
    os.environ["SF_INSTANCE_URL"] = instance_url
    return token, instance_url


SF_ACCESS_TOKEN, SF_INSTANCE_URL = request_sf_access_token()
print("✓ Salesforce JWT auth ok")
print(f"Instance URL: {SF_INSTANCE_URL}")


✓ Salesforce JWT auth ok
Instance URL: https://force-enterprise-762-dev-ed.scratch.my.salesforce.com


In [4]:
def sf_request(path: str, method: str = "GET", **kwargs):
    """Minimal helper for REST calls into the scratch org."""
    base = SF_INSTANCE_URL.rstrip("/")
    if path.startswith("http://") or path.startswith("https://"):
        url = path
    else:
        if not path.startswith("/"):
            path = "/" + path
        url = base + path

    headers = kwargs.pop("headers", {})
    headers.setdefault("Authorization", f"Bearer {SF_ACCESS_TOKEN}")
    headers.setdefault("Content-Type", "application/json")

    with httpx.Client(timeout=30) as client:
        resp = client.request(method, url, headers=headers, **kwargs)
        resp.raise_for_status()
        if not resp.text:
            return None
        try:
            return resp.json()
        except json.JSONDecodeError:
            return resp.text


def soql_query(soql: str) -> list[dict]:
    encoded = urllib.parse.quote(soql)
    first = sf_request(f"/services/data/v{SF_API_VERSION}/query?q={encoded}")
    records = list(first.get("records", []))
    next_url = first.get("nextRecordsUrl")
    while next_url:
        page = sf_request(next_url)
        records.extend(page.get("records", []))
        next_url = page.get("nextRecordsUrl")
    return records

print("✓ sf_request and soql_query ready")


✓ sf_request and soql_query ready


## Ask the LLM for an overview

This cell pulls a small snapshot of Accounts and asks the model for a concise summary.

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Fetch a snapshot of RenoCrypt accounts
accounts = soql_query(
    "SELECT Id, Name, Website, Industry, ARR__c, MRR__c, HealthScore__c, Segment__c, Support_Tier__c "
    "FROM Account ORDER BY CreatedDate DESC LIMIT 20"
)

# Trim to the most relevant fields before sending to the model
accounts_brief = [
    {
        "Id": a["Id"],
        "Name": a.get("Name"),
        "Website": a.get("Website"),
        "Industry": a.get("Industry"),
        "ARR__c": a.get("ARR__c"),
        "MRR__c": a.get("MRR__c"),
        "HealthScore__c": a.get("HealthScore__c"),
        "ChurnRisk__c": a.get("ChurnRisk__c"),
        "Segment__c": a.get("Segment__c"),
        "Support_Tier__c": a.get("Support_Tier__c"),
    }
    for a in accounts
]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You analyze SaaS customer data. Output only the requested format with no preamble, introduction, or conclusion.",
        ),
        (
            "human",
            "Analyze this Salesforce account data:\n\n"
            "<accounts_data>\n{accounts_json}\n</accounts_data>\n\n"
            "Output exactly 5-10 bullet points covering:\n"
            "- Customer segmentation (Enterprise/Mid-Market/SMB distribution)\n"
            "- Revenue tiers (ARR/MRR patterns)\n"
            "- Health score analysis (strong vs at-risk accounts)\n"
            "- Support tier alignment with revenue\n"
            "- Missing data or anomalies\n\n"
            "Output ONLY the bullet points. No text before or after.",
        ),
    ]
)

chain = prompt | llm | StrOutputParser()

overview = chain.invoke({"accounts_json": json.dumps(accounts_brief, default=str)})
print(overview)


- 4/8 (50%) active accounts are Enterprise, 2/8 Mid-Market, 1/8 SMB (excluding sample account with null segmentation)  
- Revenue tiers show strong performers: Northern Lights Bank ($420k ARR) and Acme Health ($300k ARR). 2 accounts (Everest Insurance, CloudNova) show $0 ARR/MRR  
- HealthScore: 3 accounts (Acme Health 85, Northern Lights 78, UrbanGrid 82) >75 = healthy. 3 accounts (Vertex Mfg 58, BlueSky 65, Greenfield 72) <75 = risk candidates  
- Support Tier alignment: Gold-tier accounts (Acme, Northern Lights, UrbanGrid) drive 64% of total ARR. Anomaly: Vertex Mfg (Enterprise, $240k ARR) has Bronze support despite revenue  
- Data gaps: Everest Insurance/CloudNova show $0 ARR/MRR+0 HealthScore (likely unonboarded). All accounts lack ChurnRisk data. Sample account has null values across all metrics


Loaded 30 tasks (18 open, 10 overdue)
Analyzing with LLM...


- **Task subjects/themes**: Dominant themes include technical implementation (API settings, alert routing), migration/rollout planning, audit/compliance (findings, evidence), business operations (renewals, pricing proposals), and clustered "Quarterly business review" tasks (7x in December 2025).  
- **Status breakdown**: 60% open (18/30 tasks), including 10 overdue items; 40% closed (12/30 tasks all marked "Completed").  
- **Overdue task analysis**: 10 overdue tasks (33% of total), including 7 High-priority items (e.g., "Send updated pricing proposal," "Document current incident process") and 3 Normal-priority (e.g., "Align on SecOps coverage").  
- **Priority distribution**: High-priority dominates (15 tasks, 50%), followed by Normal (14 tasks, 47%), and a single Low-priority task ("Recommend alert defaults").  
- **Critical bottlenecks**: 7/10 overdue tasks are High-priority (e.g., "Investigate metrics latency," "Confirm 

In [11]:
# BY CLAUDE 4.5 Sonnet
from datetime import datetime, date

# Fetch Tasks from Salesforce (removed Type field which may not exist)
tasks = soql_query(
    "SELECT Id, Subject, Status, ActivityDate, IsClosed, Priority "
    "FROM Task "
    "ORDER BY ActivityDate ASC NULLS LAST LIMIT 100"
)

# Calculate today's date for overdue check
today = date.today()

# Enrich tasks with overdue status
tasks_enriched = []
for task in tasks:
    task_data = {
        "Id": task["Id"],
        "Subject": task.get("Subject"),
        "Status": task.get("Status"),
        "ActivityDate": task.get("ActivityDate"),
        "IsClosed": task.get("IsClosed"),
        "Priority": task.get("Priority"),
    }
    
    # Calculate if overdue
    if task.get("ActivityDate") and not task.get("IsClosed"):
        activity_date = datetime.strptime(task["ActivityDate"], "%Y-%m-%d").date()
        task_data["IsOverdue"] = activity_date < today
    else:
        task_data["IsOverdue"] = False
    
    tasks_enriched.append(task_data)

# Count overdue tasks
overdue_count = sum(1 for t in tasks_enriched if t["IsOverdue"])
total_open = sum(1 for t in tasks_enriched if not t["IsClosed"])

print(f"Loaded {len(tasks_enriched)} tasks ({total_open} open, {overdue_count} overdue)")
print(f"Analyzing with LLM...\n")

# Prompt for task analysis
task_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You analyze CRM task data. Output only the requested format with no preamble, introduction, or conclusion.",
        ),
        (
            "human",
            "Analyze this Salesforce task data:\n\n"
            "<task_data>\n{tasks_json}\n</task_data>\n\n"
            "Summary context:\n"
            "- Total tasks: {total_tasks}\n"
            "- Open tasks: {open_tasks}\n"
            "- Overdue tasks: {overdue_tasks}\n\n"
            "Output exactly 6-10 bullet points covering:\n"
            "- Task subjects/themes (categorize by content)\n"
            "- Status breakdown (open vs closed)\n"
            "- Overdue task analysis (count, subjects, priorities)\n"
            "- Priority distribution\n"
            "- Notable patterns or risks\n\n"
            "Output ONLY the bullet points. No text before or after.",
        ),
    ]
)

task_chain = task_prompt | llm | StrOutputParser()

task_analysis = task_chain.invoke({
    "tasks_json": json.dumps(tasks_enriched[:50], default=str),  # Limit to 50 for token efficiency
    "total_tasks": len(tasks_enriched),
    "open_tasks": total_open,
    "overdue_tasks": overdue_count,
})

print(task_analysis)

Loaded 30 tasks (18 open, 10 overdue)
Analyzing with LLM...



- **Task subjects/themes**: Planning/Collaboration (onboarding, rollout planning, workshops), Technical/Operations (API settings, alert routing, metrics tuning), Review/Audit (contract sign-off, audit findings, SLO targets), Routine Business Reviews (multiple quarterly reviews)  
- **Status breakdown**: 12 completed tasks (40%), 18 open tasks (60% - including 6 "In Progress", 12 "Not Started")  
- **Overdue tasks**: 10 total overdue (56% of open tasks) - Key subjects: Pricing proposal, SecOps coverage, legacy server guidance, metrics latency investigation, SLO confirmation, incident process documentation  
- **Overdue priority mix**: 7 High (e.g., "Review renewal scope", "Investigate metrics latency"), 3 Normal (e.g., "Align on SecOps coverage", "Gather DevOps requirements")  
- **Priority distribution**: High (20 tasks, 67%), Normal (9 tasks, 30%), Low (1 task, 3%)  
- **Risk areas**: 8 overdue High-priority technical/oper

In [ ]:
# Gemini 3.0 Pro
from datetime import datetime

# 1. Define Current Date (simulated per user context)
current_date = "2025-11-27"

# 2. Fetch Task Data
# We want specific fields to allow the LLM to determine overdue status and themes
tasks = soql_query(
    "SELECT Id, Subject, Status, Priority, ActivityDate, Description, Who.Name, What.Name "
    "FROM Task "
    "ORDER BY ActivityDate ASC "
    "LIMIT 50"
)

# 3. Pre-calc overdue count for quick validation
total_tasks = len(tasks)
open_tasks = [t for t in tasks if t['Status'] != 'Completed']
overdue_tasks = [
    t for t in open_tasks 
    if t['ActivityDate'] and t['ActivityDate'] < current_date
]

print(f"Loaded {total_tasks} tasks ({len(open_tasks)} open, {len(overdue_tasks)} overdue)")
print("Analyzing with LLM...\n")

# 4. Prepare Data for LLM
# Minimize token usage by sending only necessary fields
tasks_context = [
    {
        "Subject": t.get("Subject"),
        "Status": t.get("Status"),
        "Priority": t.get("Priority"),
        "DueDate": t.get("ActivityDate"),
        "Description": t.get("Description")
    }
    for t in tasks
]

# 5. Construct Prompt
prompt_task_analysis = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a Senior Sales Operations Analyst. Your job is to identify risks and bottlenecks in workflow."
    ),
    (
        "human",
        "Current Date: {current_date}\n\n"
        "Analyze the following Salesforce Task data:\n"
        "<tasks>\n{tasks_json}\n</tasks>\n\n"
        "Provide a professional executive summary (bullet points) assessing:\n"
        "1. **Task subjects/themes**: Group tasks into high-level categories (e.g., Audit, Sales, Technical).\n"
        "2. **Status breakdown**: Brief count of Open vs Completed.\n"
        "3. **Overdue task analysis**: specific count of overdue items (DueDate < Current Date). Highlight the most critical overdue items (High Priority).\n"
        "4. **Priority distribution**: Breakdown of urgency.\n"
        "5. **Critical bottlenecks**: Identify specific high-priority tasks that are stalled.\n\n"
        "Format the output as concise Markdown bullet points."
    )
])

# 6. Chain and Invoke
chain_tasks = prompt_task_analysis | llm | StrOutputParser()

analysis = chain_tasks.invoke({
    "current_date": current_date,
    "tasks_json": json.dumps(tasks_context, default=str)
})

print(analysis)

Loaded 30 tasks (18 open, 10 overdue)
Analyzing with LLM...



```markdown
### Salesforce Task Analysis - Executive Summary

1. **Task Subjects/Themes**  
   - **Audits & Compliance**: Present audit findings, Prepare audit evidence, Review API rate limit settings  
   - **Renewals & Contracts**: Finalize contract signatures, Review renewal scope, Send updated pricing proposal  
   - **Technical Operations**: Analyze dashboard queries, Test alert routing, Tune synthetic checks, Investigate metrics latency  
   - **Workshops & Planning**: Schedule onboarding workshop, Plan phased rollout, Schedule GenAI workshop  
   - **SRE/DevOps Initiatives**: Define SRE engagement scope, Define pilot success metrics, Gather DevOps requirements  
   - **Incident Management**: Document current incident process, Review pilot KPIs  
   - **Business Reviews**: Quarterly business reviews (6x, multiple stakeholders)  

2. **Status Breakdown**  
   - **Completed**: 16 tasks  
   - **Open**:  
     - *In Prog

In [ ]:
# ChatGPT 5.1 Code Max
import json
from datetime import date
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Fresh task snapshot (independent of the cell above)
task_rows = soql_query(
    "SELECT Id, Subject, Status, ActivityDate, IsClosed, Priority "
    "FROM Task ORDER BY ActivityDate ASC NULLS LAST LIMIT 80"
)

today = date.today()
open_cnt = 0
overdue_cnt = 0
tasks_payload = []

for t in task_rows:
    is_closed = bool(t.get("IsClosed"))
    activity = t.get("ActivityDate")
    is_overdue = False
    if activity and not is_closed:
        try:
            is_overdue = date.fromisoformat(activity) < today
        except ValueError:
            is_overdue = False
    if not is_closed:
        open_cnt += 1
    if is_overdue:
        overdue_cnt += 1
    tasks_payload.append(
        {
            "Id": t.get("Id"),
            "Subject": t.get("Subject"),
            "Status": t.get("Status"),
            "Priority": t.get("Priority"),
            "ActivityDate": activity,
            "IsClosed": is_closed,
            "IsOverdue": is_overdue,
        }
    )

prompt_tasks = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a Salesforce productivity analyst. Given task records, surface concise, actionable insights.",
        ),
        (
            "human",
            "Task data (JSON):\n{tasks_json}\n\n"
            "Counts: total={total_tasks}, open={open_tasks}, overdue={overdue_tasks}.\n"
            "Return 6-8 bullet points covering: themes/subjects, status mix, overdue items (mention subjects/priorities), "
            "priority distribution, and quick risk/next-step notes. Keep bullets tight; no preamble or closing text.",
        ),
    ]
)

chain_tasks_v2 = prompt_tasks | llm | StrOutputParser()

task_summary = chain_tasks_v2.invoke(
    {
        "tasks_json": json.dumps(tasks_payload[:60], default=str),
        "total_tasks": len(tasks_payload),
        "open_tasks": open_cnt,
        "overdue_tasks": overdue_cnt,
    }
)

print(task_summary)



- **Operational Themes**: 7 clustered "Quarterly business review" tasks (Dec 2025) and technical focuses like SRE/SecOps workshops, system migration, and audit prep  
- **Status Imbalance**: 60% open tasks (18/30), including 10 overdue; 40% completed (12/30)  
- **Critical Overdues**: 6 High-priority tasks overdue (e.g., "Send updated pricing proposal", "Investigate metrics latency", "Prepare audit evidence") plus 4 Normal-priority (e.g., "Align on SecOps coverage")  
- **Priority Skew**: High-priority dominates (43% of all tasks), with 6/10 overdue items being High-priority  
- **Cluster Risk**: 7 "Quarterly business review" tasks (+1 GenAI workshop) concentrated in Dec 2025–Jan 2026, risking deadline collisions  
- **Immediate Risks**: 3 overdue High-priority planning tasks ("Define pilot success metrics", "Confirm SLO targets", "Document incident process") potentially blocking Q4 initiatives  
- **Next Steps**: Urgently address overdue High-priority items; preemptively schedule Dec

In [14]:
# ONE MASSIVE CHAIN: prompt1|llm|parser|prompt2|llm|parser|prompt3|llm|parser...
import json
from datetime import date
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# Fetch task data
task_rows = soql_query(
    "SELECT Id, Subject, Status, ActivityDate, IsClosed, Priority "
    "FROM Task ORDER BY ActivityDate ASC NULLS LAST LIMIT 80"
)

# Calculate metrics
today = date.today()
open_cnt = 0
overdue_cnt = 0
tasks_payload = []

for t in task_rows:
    is_closed = bool(t.get("IsClosed"))
    activity = t.get("ActivityDate")
    is_overdue = False
    if activity and not is_closed:
        try:
            is_overdue = date.fromisoformat(activity) < today
        except ValueError:
            is_overdue = False
    if not is_closed:
        open_cnt += 1
    if is_overdue:
        overdue_cnt += 1
    tasks_payload.append(
        {
            "Id": t.get("Id"),
            "Subject": t.get("Subject"),
            "Status": t.get("Status"),
            "Priority": t.get("Priority"),
            "ActivityDate": activity,
            "IsClosed": is_closed,
            "IsOverdue": is_overdue,
        }
    )

print(f"📊 Loaded {len(tasks_payload)} tasks | {open_cnt} open | {overdue_cnt} overdue\n")

# =============================================================================
# ONE GIANT CHAIN - All Steps Piped Together!
# =============================================================================

mega_chain = (
    # STEP 1: Extract Themes
    ChatPromptTemplate.from_messages([
        ("system", "Extract task themes. Output only theme list."),
        ("human", "<tasks>{tasks_json}</tasks>\nList 4-6 themes. Output ONLY bullet points.")
    ])
    | llm 
    | StrOutputParser()
    
    # Transform for Step 2
    | RunnableLambda(lambda themes: {"themes": themes})
    
    # Transform for Step 2 (REMOVED - passing direct string)
    # | RunnableLambda(lambda themes: {"themes": themes})
    
    # STEP 2: Analyze Priority Distribution  
    | ChatPromptTemplate.from_messages([
        ("system", "Analyze task priority patterns."),
        ("human", "<themes>{themes}</themes>\nAnalyze priority distribution patterns. Output ONLY bullet points.")
    ])
    | llm
    | StrOutputParser()
    
    # Transform for Step 3
    | RunnableLambda(lambda priorities: {"priorities": priorities})
    
    # STEP 3: Identify Overdue Patterns
    | ChatPromptTemplate.from_messages([
        ("system", "Identify overdue task patterns."),
        ("human", "<priorities>{priorities}</priorities>\nIdentify overdue patterns and risks. Output ONLY bullet points.")
    ])
    | llm
    | StrOutputParser()
    
    # Transform for Step 4
    | RunnableLambda(lambda overdue: {"overdue_patterns": overdue})
    
    # STEP 4: Synthesize Critical Problems
    | ChatPromptTemplate.from_messages([
        ("system", "Synthesize critical business problems."),
        ("human", "<overdue_patterns>{overdue_patterns}</overdue_patterns>\nSynthesize 3-4 critical problems. Output ONLY bullet points.")
    ])
    | llm
    | StrOutputParser()
    
    # Transform for Step 5
    | RunnableLambda(lambda problems: {"problems": problems})
    
    # STEP 5: Assess Business Impact
    | ChatPromptTemplate.from_messages([
        ("system", "Assess business impact of problems."),
        ("human", "<problems>{problems}</problems>\nAssess business impact (revenue, compliance, efficiency). Output ONLY bullet points.")
    ])
    | llm
    | StrOutputParser()
    
    # Transform for Step 6
    | RunnableLambda(lambda impacts: {"impacts": impacts})
    
    # STEP 6: Identify Root Causes
    | ChatPromptTemplate.from_messages([
        ("system", "Identify systemic root causes."),
        ("human", "<impacts>{impacts}</impacts>\nIdentify 3-4 root causes (not symptoms). Output ONLY bullet points.")
    ])
    | llm
    | StrOutputParser()
    
    # Transform for Step 7
    | RunnableLambda(lambda root_causes: {"root_causes": root_causes})
    
    # STEP 7: Brainstorm Solutions
    | ChatPromptTemplate.from_messages([
        ("system", "Brainstorm solutions."),
        ("human", "<root_causes>{root_causes}</root_causes>\nBrainstorm 5-7 solutions. Output ONLY bullet points.")
    ])
    | llm
    | StrOutputParser()
    
    # Transform for Step 8
    | RunnableLambda(lambda solutions: {"solutions": solutions})
    
    # STEP 8: Prioritize Solutions
    | ChatPromptTemplate.from_messages([
        ("system", "Prioritize solutions by impact/effort."),
        ("human", "<solutions>{solutions}</solutions>\nRank top 4-5 by impact/effort ratio. Output ONLY numbered list.")
    ])
    | llm
    | StrOutputParser()
    
    # Transform for Step 9
    | RunnableLambda(lambda prioritized: {"prioritized_solutions": prioritized})
    
    # STEP 9: Define Success Metrics
    | ChatPromptTemplate.from_messages([
        ("system", "Define measurable KPIs."),
        ("human", "<solutions>{prioritized_solutions}</solutions>\nDefine 3-4 KPIs with target numbers. Output ONLY bullet points.")
    ])
    | llm
    | StrOutputParser()
    
    # Transform for Step 10
    | RunnableLambda(lambda metrics: {"metrics": metrics})
    
    # STEP 10: Create Executive Summary
    | ChatPromptTemplate.from_messages([
        ("system", "Create executive summary."),
        ("human", "<metrics>{metrics}</metrics>\nCreate 3-sentence executive summary: situation, impact, path forward. Output ONLY the summary.")
    ])
    | llm
    | StrOutputParser()
)

# =============================================================================
# EXECUTE THE MEGA CHAIN - ONE INVOKE CALL!
# =============================================================================

print("=" * 80)
print("EXECUTING MEGA CHAIN: 10 LLM CALLS PIPED TOGETHER")
print("=" * 80)
print("Chain structure:")
print("prompt1 | llm | parser | transform")
print("  | prompt2 | llm | parser | transform")
print("  | prompt3 | llm | parser | transform")
print("  | prompt4 | llm | parser | transform")
print("  | prompt5 | llm | parser | transform")
print("  | prompt6 | llm | parser | transform")
print("  | prompt7 | llm | parser | transform")
print("  | prompt8 | llm | parser | transform")
print("  | prompt9 | llm | parser | transform")
print("  | prompt10 | llm | parser")
print("=" * 80)
print()

# Initial input
initial_input = {
    "tasks_json": json.dumps(tasks_payload[:60], default=str)
}

# ONE INVOKE CALL - RUNS ALL 10 STEPS!
print("Running chain... (this will take a minute as it calls the LLM 10 times)\n")
executive_summary = mega_chain.invoke(initial_input)

print("=" * 80)
print("FINAL OUTPUT: EXECUTIVE SUMMARY")
print("=" * 80)
print(executive_summary)
print()
print("✓ Successfully completed 10-step chained analysis!")

📊 Loaded 30 tasks | 18 open | 10 overdue

EXECUTING MEGA CHAIN: 10 LLM CALLS PIPED TOGETHER
Chain structure:
prompt1 | llm | parser | transform
  | prompt2 | llm | parser | transform
  | prompt3 | llm | parser | transform
  | prompt4 | llm | parser | transform
  | prompt5 | llm | parser | transform
  | prompt6 | llm | parser | transform
  | prompt7 | llm | parser | transform
  | prompt8 | llm | parser | transform
  | prompt9 | llm | parser | transform
  | prompt10 | llm | parser

Running chain... (this will take a minute as it calls the LLM 10 times)

FINAL OUTPUT: EXECUTIVE SUMMARY


**Situation:** Our IT strategy proactively targets technical debt reduction, system obsolescence mitigation, and resilience compliance to sustain critical operations. **Impact:** By enforcing strict modernization triggers and allocating dedicated resources, we minimize operational risk while exceeding 99.95% uptime for resilience-priority systems. **Path forward:** Continued adherence to quantified benchm